In [28]:
import numpy as np
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt
import keras
from shutil import copyfile
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [5]:
os.makedirs('train/cats')
os.makedirs('train/dogs')
os.makedirs('validation/cats')
os.makedirs('validation/dogs')

In [17]:
dogs = [filename for filename in os.listdir('train1') if filename.startswith("dog")]

In [18]:
cats = [filename for filename in os.listdir('train1') if filename.startswith("cat")]

In [19]:
for i in range(1000):
    copyfile('train1/' + dogs[i], 'train/dogs/' + dogs[i])

In [20]:
for i in range(1000):
    copyfile('train1/' + cats[i], 'train/cats/' + cats[i])

In [22]:
for i in range(1000, 1400):
    copyfile('train1/' + dogs[i], 'validation/dogs/' + dogs[i])

In [23]:
for i in range(1000, 1400):
    copyfile('train1/' + cats[i], 'validation/cats/' + cats[i])

In [32]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'train'
validation_data_dir = 'validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 50s 396ms/step - loss: 0.7345 - acc: 0.5305 - val_loss: 0.6670 - val_acc: 0.5387
Epoch 2/50
125/125 [==============================] - 49s 391ms/step - loss: 0.6819 - acc: 0.5930 - val_loss: 0.6435 - val_acc: 0.6250
Epoch 3/50
125/125 [==============================] - 54s 432ms/step - loss: 0.6438 - acc: 0.6385 - val_loss: 0.6181 - val_acc: 0.6725
Epoch 4/50
125/125 [==============================] - 54s 433ms/step - loss: 0.6247 - acc: 0.6485 - val_loss: 0.5918 - val_acc: 0.7037
Epoch 5/50
125/125 [==============================] - 47s 376ms/step - loss: 0.6241 - acc: 0.6640 - val_loss: 0.5856 - val_acc: 0.6975
Epoch 6/50
125/125 [==============================] - 46s 370ms/step - loss: 0.5959 - acc: 0.6895 - val_loss: 0.5985 - val_acc: 0.6837
Epoch 7/50
125/125 [==============================] - 47s 377ms/step - loss: 0.5839 - acc: 0.7055 - val_los

ImportError: `save_weights` requires h5py.